# Code for Wyndor's Production Problem

### Wyndor's Example:
\begin{equation}
\begin{split}
\max~ & 3x_1 + 5x_2 \\
s.t. ~& x_1 \leq 4\\
& 2x_2\leq 12 \\
& 3x_1 + 2x_2 \leq 18\\
& x_1,x_2\geq 0.
\end{split}
\end{equation}

In [1]:
from gurobipy import *

m = Model("Wyndor")

# Creat variables
# addVar(lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS（variable type is CONTINUOUS）, name="", column=None)
# lb: lower bound, ub: upper bound
# vtype: continuous, binary or integer
# name: name for the variable
x1 = m.addVar(name = "x1")
x2 = m.addVar(name = "x2")

# Set objective
# setObjective ( expr, sense=None )
# expr: linear or quadratic expression
# sense: GRB.MINIMIZE or GRB.MAXIMIZE
m.setObjective(3*x1 + 5*x2, GRB.MAXIMIZE)

# Add constraint: 
m.addConstr(x1 <= 4, "Plant1")

m.addConstr(2*x2 <= 12, "Plant2")

m.addConstr(3*x1 + 2*x2 <= 18, "Plant3")

# Solving the model
m.optimize()

#  Print optimal solutions and optimal value
for v in m.getVars():
    print(v.VarName,":", v.x)
    
print('Obj:', m.objVal)



Academic license - for non-commercial use only
Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.13s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.24 seconds
Optimal objective  3.600000000e+01
x1 : 2.0
x2 : 6.0
Obj: 36.0


## Separating Model and Data

In [2]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

# Objective coefficient: profit for each product
profit = np.array([3, 5])
print(profit)
print(type(profit))
print(profit.shape)

[3 5]
<class 'numpy.ndarray'>
(2,)


In [3]:
# Constraint right-hand-side: capacity for each plant
capacity = np.array([4, 12, 18])

# A matrix: the consumption of capacity at plant i of product j
rate = np.array([[1, 0],
                 [0, 2], 
                 [3, 2]])

print(rate)
print(rate.shape)

[[1 0]
 [0 2]
 [3 2]]
(3, 2)


In [4]:
# From A matrix, extract the number of products: N and the number of plants: M
M, N = rate.shape

print("M = %g,  N = %g" % (M, N) )

M = 3,  N = 2


In [5]:
#########Model Set-up###############

m = Model("Wyndor")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )
# indices: can be one or more integer values, e.g.,  x = model.addVars(2), which creates x[0], x[1]; 
#          e.g., x = model.addVars(2, 3), which creates x[0,0],x[0,1],x[0,2],x[1,0],x[1,1],x[1,2];
#          can be a list of tuples, e.g., x = model.addVars([(0,0), (1,1), (2,2)]), which creates x[0,0],x[1,1],x[2,2].

x = m.addVars(N, name = "x")

# Set objective
#quicksum is recommened in gurobi
m.setObjective( quicksum(profit[i]*x[i] for i in range(N)), GRB.MAXIMIZE)

# Add constraints: 
m.addConstrs(( quicksum(rate[i,j]*x[j] for j in range(N)) <= capacity[i] for i in range(M) ), "Plant")

# Solving the model
m.optimize()

# Print optimal solutions and optimal value
for v in m.getVars():
    print(v.VarName, v.x)
    
print('Obj:', m.objVal)



Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.03s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.05 seconds
Optimal objective  3.600000000e+01
x[0] 2.0
x[1] 6.0
Obj: 36.0


In [6]:
print("\n Sensitivity information")
for d in m.getConstrs():
    print(d.ConstrName,d.Pi,d.SARHSUp,d.SARHSLow)


 Sensitivity information
Plant[0] 0.0 1e+100 2.0
Plant[1] 1.5 18.0 6.0
Plant[2] 1.0 24.0 12.0


### Loading Data from csv. Files

In [13]:
from gurobipy import *
import numpy as np
import pandas as pd


#########Parameters Set-up############
# Read the data from the csv file and use the first column as the index of rows
Wyndor_data = pd.read_csv('Wyndor.csv', index_col = 0)#specify the first column as index otherwise treated as data
print(Wyndor_data)
print(Wyndor_data.shape)

# Record the number of rows and columns in the data
temp_M, temp_N = Wyndor_data.shape


                  Doors  Windows  Capacity
Profit per batch      3        5       NaN
Plant 1               1        0       4.0
Plant 2               0        2      12.0
Plant 3               3        2      18.0
(4, 3)


In [14]:
#One is referred to http://pandas.pydata.org/pandas-docs/stable/indexing.html 
#for indexing and selecting data for the dataframe object in pandas.

# Extracting objective coefficient via selection by position: .iloc
# When slicing the data, the start bound is included, while the end bound is excluded
profit = Wyndor_data.iloc[0, 0:temp_N-1]
print(profit) 
print('Index of the data:')
print(profit.index) 
print('Values of the data:')
print(profit.values)

#Extracting the values by ignoring the index and header of the dataframe
profit = profit.values

Doors      3.0
Windows    5.0
Name: Profit per batch, dtype: float64
Index of the data:
Index(['Doors', 'Windows'], dtype='object')
Values of the data:
[3. 5.]


In [15]:
#Constraint right-hand-side: capacity for each plant
capacity = Wyndor_data.iloc[1:temp_M, temp_N-1]
capacity = capacity.values

#A matrix: the consumption of capacity at plant i of product j
rate = Wyndor_data.iloc[1:temp_M, 0:temp_N-1]
print(rate)
#drop the index of the data
rate = rate.values
print(rate)

#From A matrix, extract the number of products: N and the number of plants: M
M, N = rate.shape


         Doors  Windows
Plant 1      1        0
Plant 2      0        2
Plant 3      3        2
[[1 0]
 [0 2]
 [3 2]]


In [16]:
#########Model Set-up###############

m = Model("Wyndor")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )
x = m.addVars(N, name = 'Product')

# Set objective
m.setObjective( quicksum(profit[i]*x[i] for i in range(N)), GRB.MAXIMIZE)

# Add constraints: 
m.addConstrs(( quicksum(rate[i,j]*x[j] for j in range(N)) <= capacity[i] for i in range(M) ), name = "Plant")

# Solving the model
m.optimize()

#  Print optimal solutions and optimal value
for v in m.getVars():
    print(v.VarName, v.x)
    
print('Obj:', m.objVal)


Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.04s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.05 seconds
Optimal objective  3.600000000e+01
Product[0] 2.0
Product[1] 6.0
Obj: 36.0
